<a href="https://colab.research.google.com/github/rhmes/point-cloud-compression/blob/main/point_cloud_compression_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point Cloud Compression Demo

This notebook demonstrates installation, core functionality, and testing for the point cloud compression project.

In [6]:
# Connect to Google Drive (for Colab users)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive mounted.')
except ImportError:
    print('Not running in Colab, skipping Google Drive mount.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.


In [4]:
# Clone repo only if not already cloned
import os
repo_url = 'https://github.com/rhmes/point-cloud-compression.git'
parent_dir = '/MyDrive/projects'
repo_dir = os.path.join(parent_dir, 'point-cloud-compression')
os.makedirs(parent_dir, exist_ok=True)
os.chdir(parent_dir)
if not os.path.exists('point-cloud-compression'):
    os.system(f'git clone {repo_url}')
os.chdir('point-cloud-compression')
print(f'Current working directory: {os.getcwd()}')

FileNotFoundError: [Errno 2] No such file or directory: 'point-cloud-compression'

In [ ]:
# Check if venv_cpu or venv_gpu exists, run setup if not
import os
if not (os.path.exists('venv_cpu') or os.path.exists('venv_gpu')):
    print('No virtual environment found. Running setup...')
    os.system('bash venv_gpu_setup.sh')  # Change to venv_cpu_setup.sh if needed
else:
    print('Virtual environment already exists. Skipping setup.')

In [ ]:
# Download ModelNet40 and ShapeNet datasets from Google Drive (Option 1 from README)
import os
import urllib.request

def download_from_gdrive(url, output_path):
    try:
        import gdown
    except ImportError:
        os.system('pip install gdown')
        import gdown
    gdown.download(url, output_path, quiet=False)

# ModelNet40
modelnet_url = 'https://drive.google.com/uc?id=1Isa8seckZ9oNzstlE7VZcd6wVVx8LdMF'
modelnet_zip = 'ModelNet40_pc_8192.zip'
if not os.path.exists('data/ModelNet40_pc_8192'):
    print('Downloading ModelNet40 pre-converted point clouds...')
    download_from_gdrive(modelnet_url, modelnet_zip)
    import zipfile
    with zipfile.ZipFile(modelnet_zip, 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove(modelnet_zip)
    print('ModelNet40 download and extraction complete.')
else:
    print('ModelNet40 dataset already exists. Skipping download.')

# ShapeNet
shapenet_url = 'https://drive.google.com/uc?id=1OzaU01kolBpfRRD0zKESYh67Hh2s2dbD'
shapenet_zip = 'ShapeNet_pc_2048.zip'
if not os.path.exists('data/ShapeNet_pc_2048'):
    print('Downloading ShapeNet pre-converted point clouds...')
    download_from_gdrive(shapenet_url, shapenet_zip)
    import zipfile
    with zipfile.ZipFile(shapenet_zip, 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove(shapenet_zip)
    print('ShapeNet download and extraction complete.')
else:
    print('ShapeNet dataset already exists. Skipping download.')

In [ ]:
# Example core functionality: List files in the data directory

def list_data_files(data_dir='data'):
    if not os.path.exists(data_dir):
        print(f"Directory '{data_dir}' does not exist.")
        return []
    files = os.listdir(data_dir)
    print(f"Files in '{data_dir}':", files)
    return files

# Run the function
data_files = list_data_files()

In [ ]:
# Compress point cloud test files using the trained model
!python compress.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './model/K256' --K 256

In [ ]:
# Decompress the compressed point cloud files
!python decompress.py './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './model/K256' --K 256

In [ ]:
# Evaluate the compression results using PSNR, Chamfer distance, and bpp metrics
!python eval.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './eval/ModelNet40_K256.csv' '../geo_dist/build/pc_error'

In [ ]:
# Visualize evaluation metrics and save plots
!python visualize.py --csv './eval/ModelNet40_K256.csv' --outdir './figure/'